In [ ]:
# auto updates code chnages
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import os
import sys
import pynbody
import numpy as np
basepath = os.path.dirname(os.path.dirname(os.getcwd()))
if basepath not in sys.path:
    sys.path.append(basepath)
from base import *
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
sim, halos = load_sim('cptmarvel', 4096)

In [ ]:
expelled_cptmarvel_1 = load_expelled_particles('cptmarvel_1')
ejected_cptmarvel_1 = load_ejected_particles('cptmarvel_1')

In [ ]:
h = halos[1]
snap_time = h.properties['time'].in_units('Gyr')
expelled_time_mask = expelled_cptmarvel_1['time'] == snap_time
expelled_40096 = expelled_cptmarvel_1[expelled_time_mask]
ejected_time_mask = ejected_cptmarvel_1['time'] == snap_time
ejected_4096 = ejected_cptmarvel_1[ejected_time_mask]


In [ ]:
# centering the halo 
pynbody.analysis.angmom.faceon(h)


In [ ]:
# Setting parameters for the analysis
nbins = 100
delta_T = 1 # Gyr
r_min = 0.1 # kpc
sfr_window_start = snap_time - pynbody.units.Unit(f'{delta_T} Gyr').in_units('Gyr')
r_max = expelled_40096['r_gal'].iloc[0]
bins = np.logspace(np.log10(r_min), np.log10(r_max), nbins + 1)

In [ ]:
# star properties
stars = h.star
star_pos = stars['pos'].in_units('kpc')
# calculating the distance from the center of the halo
star_dist = np.sqrt(np.sum(star_pos**2, axis=1))
star_mass = stars['mass'].in_units('Msol')
star_tform = stars['tform'].in_units('Gyr')

In [ ]:
binned_sfr = np.zeros(nbins)
binned_expelled_mass = np.zeros(nbins)
binned_ejected_mass = np.zeros(nbins)

for i in range(nbins):
    # setting the bin limits
    bin_min = bins[i]
    # handling for the last bin
    bin_max = bins[i+1] 

    # selecting for in bin stars, outflows and ejected particles
    selected_stars = stars[(star_dist >= bin_min) & (star_dist < bin_max)]
    selected_expelled = expelled_40096[(expelled_40096['r'] >= bin_min) & (expelled_40096['r'] < bin_max)]
    selected_ejected = ejected_4096[(ejected_4096['r'] >= bin_min) & (ejected_4096['r'] < bin_max)]

    # selecting particles that formed in the time window
    selected_formed_stars = selected_stars[(selected_stars['tform'].in_units('Gyr') >= sfr_window_start) & (selected_stars['tform'].in_units('Gyr') <= snap_time)]

    # calculating formation rate
    selected_mass_formed = selected_formed_stars['mass'].sum().in_units('Msol')
    binned_sfr[i] = selected_mass_formed / delta_T

    # calculating outflow and ejected mass
    binned_expelled_mass[i] = selected_expelled['mass'].sum()
    binned_ejected_mass[i] = selected_ejected['mass'].sum()

binned_mlf_expelled = np.divide(binned_expelled_mass, binned_sfr, out=np.zeros_like(binned_expelled_mass), where=binned_sfr != 0)
binned_mlf_ejected = np.divide(binned_ejected_mass, binned_sfr, out=np.zeros_like(binned_ejected_mass), where=binned_sfr != 0)


In [ ]:
get

In [ ]:
# # plotting mlf

# bin_centers = 0.5 * (bins[:-1] + bins[1:])

# r_half = expelled_40096['r_half'].iloc[0]

# plt.scatter(bin_centers, binned_sfr, marker='o', color='blue')
# plt.xlabel('Radius (kpc)')
# plt.ylabel('Stellar Formation Rate (Msol/Gyr)')
# plt.axvline(x=r_half, color='black', linestyle='-')
# plt.semilogy()
# plt.show()

# plt.scatter(bin_centers, binned_mlf_ejected, marker='o', color='red')
# plt.xlabel('Radius (kpc)')
# plt.ylabel('Ejected Mass / SFR')
# plt.semilogy()
# plt.show()

# plt.scatter(bin_centers, binned_mlf_expelled, marker='o', color='green')
# plt.xlabel('Radius (kpc)')
# plt.ylabel('Mass Loading Factor')
# plt.semilogy()
# plt.show()


In [ ]:
# need to make a mass loading plot by function of halo mass
# first code to find MLF of a halo
def mlf_of_halo(halo, snap_time, delta_T=1):
    """
    Calculate the mass loading factor of a halo at a given snapshot time.

    Parameters:
    halo (pynbody.halo.Halo): The halo to analyze.
    snap_time (float): The snapshot time in Gyr.
    delta_T (float): The time window for SFR calculation in Gyr.

    Returns:
    tuple: Bin centers, binned SFR, binned expelled mass, binned ejected mass, MLF expelled, MLF ejected.
    """
    
    # Extracting star properties
    stars = halo.star
    star_pos = stars['pos'].in_units('kpc')
    star_dist = np.sqrt(np.sum(star_pos**2, axis=1))
    star_mass = stars['mass'].in_units('Msol')
    star_tform = stars['tform'].in_units('Gyr')

    # Setting parameters for the analysis
    nbins = 100
    r_min = 0.1  # kpc
    r_max = halo.properties['r_half'].in_units('kpc')
    bins = np.logspace(np.log10(r_min), np.log10(r_max), nbins + 1)

    # Initializing arrays for binned data
    binned_sfr = np.zeros(nbins)
    binned_expelled_mass = np.zeros(nbins)
    binned_ejected_mass = np.zeros(nbins)

    for i in range(nbins):
        bin_min = bins[i]
        bin_max = bins[i + 1]

        selected_stars = stars[(star_dist >= bin_min) & (star_dist < bin_max)]
        selected_expelled = expelled_cptmarvel_1[(expelled_cptmarvel_1['r'] >= bin_min) & (expelled_cptmarvel_1['r'] < bin_max)]
        selected_ejected = ejected_cptmarvel_1[(ejected_cptmarvel_1['r'] >= bin_min) & (ejected_cptmarvel_1['r'] < bin_max)]

        selected_formed_stars = selected_stars[(selected_stars['tform'].in_units('Gyr') >= snap_time - delta_T) & (selected_stars['tform'].in_units('Gyr') <= snap_time)]

        selected_mass_formed = selected_formed_stars['